In [1]:
import pickle

with open('./dataset complete/train_test_data/train_data.pkl', 'rb') as file:
    train_data = pickle.load(file)
train_inputs, train_contexts = train_data[0]
train_tagets = train_data[1]
del train_data

In [2]:
with open('./dataset complete/train_test_data/test_data.pkl', 'rb') as file:
    test_data = pickle.load(file)
test_inputs, test_contexts = test_data[0]
test_tagets = test_data[1]
del test_data

In [3]:
import json

with open ("./dataset complete/character_to_prediction_index.json", "r") as f:
    characters = json.load(f)

# TODO: tune hyperparameters
EPOCHS = 20
BATCH_SIZE=64
d_model = 128  # hidden layer(s) dimensionality
num_layers = 4 # how many encoders and decoders to stack
num_heads = 7 # how many attention heads should every mha have
ff_dim = 200 # how many neurons shall feed-forward layers have
dropout_rate = 0.1

output_vocab_size = len(characters) + 2

In [4]:
from transformer import get_compiled_transformer

transformer = get_compiled_transformer(
    d_model=d_model,
    num_layers=num_layers,
    num_heads=num_heads,
    ff_dim=ff_dim,
    dropout_rate=dropout_rate,
    output_vocab_size=output_vocab_size
)

In [5]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_checkpoints/cp-{epoch:04d}.ckpt", save_weights_only=True, verbose=1
)
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

history = transformer.fit([train_inputs, train_contexts], train_tagets, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1, callbacks=[cp_callback, es_callback])

transformer.evaluate([test_inputs, test_contexts], test_tagets, batch_size=BATCH_SIZE)

In [ ]:
print(history.history.keys())

In [1]:
import matplotlib.pyplot as plt

plt.plot(history.history['masked_accuracy'])
plt.plot(history.history['val_masked_accuracy'])
plt.title('Accuracy over Epochs')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

ImportError: DLL load failed while importing _imaging: Das angegebene Modul wurde nicht gefunden.

In [2]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss over Epochs')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'plt' is not defined